# Produce PDF report of network stations showing RF waveforms

In [ ]:
# Imports
import os
# from collections import defaultdict

import numpy as np
import rf
import rf.imaging

import tqdm.auto as tqdm

In [ ]:
import importlib
import seismic.receiver_fn.rf_util as rf_util
import seismic.receiver_fn.rf_plot_utils as rf_plot_utils
import seismic.receiver_fn.rf_stacking as rf_stacking

In [ ]:
# importlib.reload(rf_plot_utils)
# importlib.reload(rf_util)

In [ ]:
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.pyplot as plt
import matplotlib

## Read source file

In [ ]:
# rf_type = 'ZRT_td'
rf_type = 'ZRT_it'

In [ ]:
network = 'OA'
# network = '7X'

In [ ]:
src_file = r"..\DATA\OA_rfs_20170911T000036-20181128T230620_{}_rev10_qual.h5".format(rf_type)
# src_file = r"..\DATA\7X_rfs_20090616T034200-20110401T231849_{}_rev1_qual.h5".format(rf_type)
# src_file = r"..\DATA\7X_rfs_20090616T034200-20110401T231849_{}_rev1.1_qual.h5".format(rf_type)

In [ ]:
data_all = rf_util.read_h5_rf(src_file)

In [ ]:
# data_all[0].stats

## Convert to hierarchical dictionary format

In [ ]:
data_dict = rf_util.rf_to_dict(data_all)

## Load event mask files if available

In [ ]:
event_mask_path = None
# event_mask_path = r"C:\software\hiperseis\seismic\receiver_fn\DATA\event_mask_{}".format(network)

In [ ]:
event_mask_dict = None
if event_mask_path is not None and os.path.isdir(event_mask_path):
    import re
    mask_files = os.listdir(event_mask_path)
    mask_files = [f for f in mask_files if os.path.isfile(os.path.join(event_mask_path, f))]
#     print(mask_files)
    pattern = "([A-Za-z0-9\.]{5,})_event_mask\.txt"
    pattern = re.compile(pattern)
    event_mask_dict = dict()
    for f in mask_files:
        match_result = pattern.match(f)
        if not match_result:
            continue
        code = match_result[1]
#         print(code)
        with open(os.path.join(event_mask_path, f), 'r') as f:
            events = f.readlines()
            events = set([e.strip() for e in events])
            event_mask_dict[code] = events
        # end with
    # end for
# end if

if event_mask_dict:
    print("Loaded {} event masks".format(len(event_mask_dict)))
# end if

## Plot a few items interactively to check data

In [ ]:
i = 0
for st, station_db in data_dict.items():
    i += 1
    if i > 3:
        break
    channel = rf_util.choose_rf_source_channel(rf_type, station_db)
    channel_data = station_db[channel]
    full_code = '.'.join([network, st, channel])

    t_channel = list(channel)
    t_channel[-1] = 'T'
    t_channel = ''.join(t_channel)

    if event_mask_dict and full_code in event_mask_dict:
        # Select events from external source
        event_mask = event_mask_dict[full_code]
        rf_stream = rf.RFStream([tr for tr in channel_data if tr.stats.event_id in event_mask]).sort(['back_azimuth'])
    else:
        rf_util.label_rf_quality_simple_amplitude(rf_type, channel_data)
        rf_stream = rf.RFStream([tr for tr in channel_data if tr.stats.predicted_quality == 'a']).sort(['back_azimuth'])
    # end if

    # Find matching HHT channel data
    events = [tr.stats.event_id for tr in rf_stream]
    transverse_data = station_db[t_channel]
    t_stream = rf.RFStream([tr for tr in transverse_data if tr.stats.event_id in events]).sort(['back_azimuth'])

    # Plot to notebook
    fig = rf_plot_utils.plot_rf_wheel([rf_stream, t_stream], layout=(1,2))
    plt.show()
    
    fig = rf_plot_utils.plot_rf_stack(rf_stream, time_window=(-5, 25))
    print(plt.subplots_adjust())
    plt.show()
    
    fig = rf_plot_utils.plot_rf_stack(t_stream, time_window=(-5, 25))
    plt.show()

## Setup and layout functions

In [ ]:
A4_size = (8.27, 11.69)

In [ ]:
def get_aspect(ax):
    from operator import sub
    # Total figure size
    figW, figH = ax.get_figure().get_size_inches()
    # Axis size on figure
    _, _, w, h = ax.get_position().bounds
    # Ratio of display units
    disp_ratio = (figH * h) / (figW * w)
    # Ratio of data units
    # Negative over negative because of the order of subtraction
    data_ratio = sub(*ax.get_ylim()) / sub(*ax.get_xlim())

    return disp_ratio / data_ratio

In [ ]:
def rf_layout_A4(fig):

    # Fix aspect ratio of stack plot
    ax = fig.axes[3]
    ax.set_aspect(get_aspect(ax))
    
    # Fix aspect ratio of traces plot
    ax = fig.axes[0]
    ax.set_aspect(get_aspect(ax))
    
    # Set to A4 paper size
    fig.set_size_inches(*A4_size)
    
    # Adjust position of stack plot to fixed location on page
    ax3 = fig.axes[3]
    ax3_pos = ax3.get_position()
    ax3_top = 0.95
    ax3.set_position([ax3_pos.x0, ax3_top - ax3_pos.height, ax3_pos.width, ax3_pos.height])
    ax3.set_anchor('NW')        
    ax3_pos = ax3.get_position()

    # Adjust position of traces plot to pack at top of page with against stack plot
    ax0 = fig.axes[0]
    ax0_pos = ax0.get_position()
    ax0.set_position([ax0_pos.x0, ax3_pos.y0 - 0.01 - ax0_pos.height, ax0_pos.width, ax0_pos.height])
    ax0.set_anchor('NW')
    ax0_pos = ax0.get_position()
    
    # Adjust y-position of distance/azimuth marker plots to match traces plot
    ax1 = fig.axes[1]
    ax2 = fig.axes[2]
    ax1_pos = ax1.get_position()
    ax2_pos = ax2.get_position()
    ax1.set_position([ax1_pos.x0, ax0_pos.y0, ax1_pos.width, ax0_pos.height])
    ax2.set_position([ax2_pos.x0, ax0_pos.y0, ax2_pos.width, ax0_pos.height])
# end func

## Plot all data to PDF file

In [ ]:
fixed_stack_height_inches = 0.8
y_pad_inches = 1.6
total_trace_height_inches = A4_size[1] - fixed_stack_height_inches - y_pad_inches
max_trace_height = 0.2
outfile_name = '{}_RF_summary.pdf' if not event_mask_dict else '{}_RF_summary_handpicked.pdf'
with PdfPages(outfile_name.format(network)) as pdf:
    plt.rc('text', usetex=False)  # Would like to use Tex, but lack desktop PC privileges to update packages to what is required
    pbar = tqdm.tqdm(total=len(data_dict))
    for st, station_db in data_dict.items():
        pbar.update()
        pbar.set_description("{}.{}".format(network, st))

        # Choose RF channel
        channel = rf_util.choose_rf_source_channel(rf_type, station_db)
        channel_data = station_db[channel]
        full_code = '.'.join([network, st, channel])

        t_channel = list(channel)
        t_channel[-1] = 'T'
        t_channel = ''.join(t_channel)

        if event_mask_dict and full_code in event_mask_dict:
            # Select events from external source
            event_mask = event_mask_dict[full_code]
            rf_stream = rf.RFStream([tr for tr in channel_data if tr.stats.event_id in event_mask]).sort(['back_azimuth'])
        else:
            # Label and filter quality
            rf_util.label_rf_quality_simple_amplitude(rf_type, channel_data)
            rf_stream = rf.RFStream([tr for tr in channel_data if tr.stats.predicted_quality == 'a']).sort(['back_azimuth'])
        # end if
        if not rf_stream:
            continue

        # Find matching HHT channel data
        events = [tr.stats.event_id for tr in rf_stream]
        transverse_data = station_db[t_channel]
        t_stream = rf.RFStream([tr for tr in transverse_data if tr.stats.event_id in events]).sort(['back_azimuth'])
        if not t_stream:
            continue

        # Plot pinwheel of primary and transverse components
        fig = rf_plot_utils.plot_rf_wheel([rf_stream, t_stream], fontscaling=0.8)
        fig.set_size_inches(*A4_size)
        plt.tight_layout()
        plt.subplots_adjust(hspace=0.15, top=0.95, bottom=0.15)
        ax = fig.gca()
        fig.text(-0.32, -0.32, "\n".join(rf_stream[0].stats.processing), fontsize=6, transform=ax.transAxes)
        pdf.savefig(dpi=300, papertype = 'a4', orientation = 'portrait')
        plt.close()

        num_traces = len(rf_stream)
        assert len(t_stream) == num_traces

        # Plot RF stack of primary component
        trace_ht = min(total_trace_height_inches/num_traces, max_trace_height)
        fig = rf_plot_utils.plot_rf_stack(rf_stream, trace_height=trace_ht, stack_height=fixed_stack_height_inches, fig_width=A4_size[0])
        fig.suptitle("Channel {}".format(rf_stream[0].stats.channel))
        # Customize layout to pack to top of page while preserving RF plots aspect ratios
        rf_layout_A4(fig)
        # Save to new page in file
        pdf.savefig(dpi=300, papertype='a4', orientation='portrait')
        plt.close()

        # Plot RF stack of transverse component
        fig = rf_plot_utils.plot_rf_stack(t_stream, trace_height=trace_ht, stack_height=fixed_stack_height_inches, fig_width=A4_size[0])
        fig.suptitle("Channel {}".format(t_stream[0].stats.channel))
        # Customize layout to pack to top of page while preserving RF plots aspect ratios
        rf_layout_A4(fig)
        # Save to new page in file
        pdf.savefig(dpi=300, papertype='a4', orientation='portrait')
        plt.close()
    # end for
    pbar.close()
# end with